In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rc
from matplotlib.ticker import LogLocator
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

FOR_PRINT = True

if FOR_PRINT:
    LINE_WIDTH = 1
    MARKER_SIZE = 3
    FONT_SIZE = 8

    AXES_WIDTH = 0.65 * LINE_WIDTH

    plt.rcParams['grid.linewidth']=AXES_WIDTH
    plt.rcParams['axes.linewidth']=AXES_WIDTH
    plt.rcParams['axes.labelpad']=3.0

    plt.rcParams['xtick.major.pad']=0
    plt.rcParams['xtick.major.size']=2.0
    plt.rcParams['xtick.major.width']=AXES_WIDTH
    plt.rcParams['xtick.minor.size']=1.0
    plt.rcParams['xtick.minor.width']=0.75 * AXES_WIDTH

    plt.rcParams['ytick.major.pad']=-1.5
    plt.rcParams['ytick.major.size']=2.0
    plt.rcParams['ytick.major.width']=AXES_WIDTH
    plt.rcParams['ytick.minor.size']=1.0
    plt.rcParams['ytick.minor.width']=0.75 * AXES_WIDTH
else:
    LINE_WIDTH = 6
    MARKER_SIZE = 14
    FONT_SIZE = 45

%matplotlib inline
#plt.rcParams['figure.figsize'] = [15, 15]
plt.rcParams['lines.linewidth'] = LINE_WIDTH
plt.rcParams['lines.markeredgewidth'] = 0.75 * LINE_WIDTH
plt.rcParams['lines.markersize'] = MARKER_SIZE
plt.rcParams['font.size'] = FONT_SIZE
rc('text', usetex=True)

In [ ]:
data = list()
with open("hex_results.json") as results_json_file:
    data = json.load(results_json_file)

In [ ]:
def draw_convergence_triangle(fig, ax, origin, width_inches, slope, inverted=False, color=None, polygon_kwargs=None, label=True, labelcolor=None, label_kwargs=None, zorder=None):
    """
    This function draws slopes or "convergence triangles" into loglog plots.

    @param fig: The figure
    @param ax: The axes object to draw to
    @param origin: The 2D origin (usually lower-left corner) coordinate of the triangle
    @param width_inches: The width in inches of the triangle
    @param slope: The slope of the triangle, i.e. order of convergence
    @param inverted: Whether to mirror the triangle around the origin, i.e. whether 
        it indicates the slope towards the lower left instead of upper right (defaults to false)
    @param color: The color of the of the triangle edges (defaults to default color)
    @param polygon_kwargs: Additional kwargs to the Polygon draw call that creates the slope
    @param label: Whether to enable labeling the slope (defaults to true)
    @param labelcolor: The color of the slope labels (defaults to the edge color)
    @param label_kwargs: Additional kwargs to the Annotation draw call that creates the labels
    @param zorder: The z-order value of the triangle and labels, defaults to a high value
    """

    if polygon_kwargs is None:
        polygon_kwargs = {}
    if label_kwargs is None:
        label_kwargs = {}

    if color is not None:
        polygon_kwargs["color"] = color
    if "linewidth" not in polygon_kwargs:
        polygon_kwargs["linewidth"] = 0.75 * mpl.rcParams["lines.linewidth"]
    if labelcolor is not None:
        label_kwargs["color"] = labelcolor
    if "color" not in label_kwargs:
        label_kwargs["color"] = polygon_kwargs["color"]
    if "fontsize" not in label_kwargs:
        label_kwargs["fontsize"] = 0.75 * mpl.rcParams["font.size"]

    if inverted:
        width_inches = -width_inches
    if zorder is None:
        zorder = 10

    # For more information on coordinate transformations in Matplotlib see
    # https://matplotlib.org/3.1.1/tutorials/advanced/transforms_tutorial.html

    # Convert the origin into figure coordinates in inches
    origin_disp = ax.transData.transform(origin)
    origin_dpi = fig.dpi_scale_trans.inverted().transform(origin_disp)

    # Obtain the bottom-right corner in data coordinates
    corner_dpi = origin_dpi + width_inches * np.array([1.0, 0.0])
    corner_disp = fig.dpi_scale_trans.transform(corner_dpi)
    corner = ax.transData.inverted().transform(corner_disp)

    (x1, y1) = (origin[0], origin[1])
    x2 = corner[0]

    # The width of the triangle in data coordinates
    width = x2 - x1
    # Compute offset of the slope
    log_offset = y1 / (x1 ** slope)

    y2 = log_offset * ((x1 + width) ** slope)
    height = y2 - y1

    # The vertices of the slope
    a = origin
    b = corner
    c = [x2, y2]

    # Draw the slope triangle
    X = np.array([a, b, c])
    triangle = plt.Polygon(X[:3,:], fill=False, zorder=zorder, **polygon_kwargs)
    ax.add_patch(triangle)

    # Convert vertices into display space
    a_disp = ax.transData.transform(a)
    b_disp = ax.transData.transform(b)
    c_disp = ax.transData.transform(c)

    # Figure out the center of the triangle sides in display space
    bottom_center_disp = a_disp + 0.5 * (b_disp - a_disp)
    bottom_center = ax.transData.inverted().transform(bottom_center_disp)

    right_center_disp = b_disp + 0.5 * (c_disp - b_disp)
    right_center = ax.transData.inverted().transform(right_center_disp)

    # Label alignment depending on inversion parameter
    va_xlabel = "top" if not inverted else "bottom"
    ha_ylabel = "left" if not inverted else "right"

    # Label offset depending on inversion parameter
    offset_xlabel = [0.0, -0.33 * label_kwargs["fontsize"]] if not inverted else [0.0, 0.33 * label_kwargs["fontsize"]]
    offset_ylabel = [0.33 * label_kwargs["fontsize"], 0.0] if not inverted else [-0.33 * label_kwargs["fontsize"], 0.0]

    # Draw the slope labels
    ax.annotate("$1$", bottom_center, xytext=offset_xlabel, textcoords='offset points', ha="center", va=va_xlabel, zorder=zorder, **label_kwargs)
    ax.annotate(f"${slope}$", right_center, xytext=offset_ylabel, textcoords='offset points', ha=ha_ylabel, va="center", zorder=zorder, **label_kwargs)


In [ ]:
FIG_WIDTH = 3.6 if FOR_PRINT else 20
FIG_HEIGHT = 2.25 if FOR_PRINT else 12
SLOPE_WIDTH = 0.125 * FIG_WIDTH

fig = plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))

def method_sort_order_key(method_name):
    method_mapping = {
        "fem_hex20": 1,
        "fcm_hex20": 3,
        "fem_hex8": 0,
        "fcm_hex8": 2
    }
    return method_mapping[method_name]

methods = data['methods']
method_names = sorted([method for method in methods], key=method_sort_order_key)


def get_label(method):
    method_mapping = {
        "fem_hex20": "FEM Hex20",
        "fcm_hex20": "FCM Hex20",
        "fem_hex8": "FEM Hex8",
        "fcm_hex8": "FCM Hex8"
    }
    return method_mapping[method]

for method in method_names:
    method_data = methods[method]
    resolutions = [entry['resolution'] for entry in method_data]
    l2_errors = [entry['l2_error'] for entry in method_data]
    mesh_sizes = [entry['mesh_size'] for entry in method_data]
    
    plt.plot(mesh_sizes, l2_errors, '-o', label=get_label(method))
    
plt.legend(prop={'size': 0.75 * FONT_SIZE}, loc = 'lower right')
plt.grid()
plt.xlabel('Cell width $h$', fontsize=FONT_SIZE)
plt.ylabel('$L^2$ error', fontsize=FONT_SIZE)
plt.loglog()

plt.xlim(2e-2, 1.5e0)
plt.ylim(1e-6, 1e-1)

plt.axes().yaxis.set_major_locator(LogLocator(10.0, subs=(1.0,)))
plt.tick_params(axis='both', which='major', labelsize=FONT_SIZE)

plt.grid(which='major', linestyle='-', linewidth=0.75 * LINE_WIDTH)
#plt.grid(axis='x', which='minor', linestyle='--', linewidth=0.25 * LINE_WIDTH, color="lightgray")
plt.grid(which='minor', linestyle='--', linewidth=0.25 * LINE_WIDTH, color="lightgray")

# Whether to use 
use_single_color_slopes = True
single_color_slopes = "dimgrey"

color_per_slope = {3: "tab:red", 2: "tab:green", 1: "tab:blue"}
if use_single_color_slopes:
    color_per_slope = {s: single_color_slopes for s,c in color_per_slope.items()}

ax = plt.gca()
draw_convergence_triangle(fig, ax, [2.00e-1, 2.5e-4], SLOPE_WIDTH, 3, color=color_per_slope[3])
draw_convergence_triangle(fig, ax, [7.00e-2, 5.25e-4], SLOPE_WIDTH, 2, color=color_per_slope[2])
draw_convergence_triangle(fig, ax, [2.60e-1, 2.0e-2], SLOPE_WIDTH, 1, color=color_per_slope[1], inverted=True)

plt.tight_layout()
#plt.savefig('convergence_rate.pdf', format='pdf', bbox_inches='tight')
plt.savefig('convergence_rate.pgf', format='pgf', bbox_inches='tight')
plt.show()

